<a href="https://colab.research.google.com/github/RebeccaKessler/Machine_Learning/blob/main/Codes/CamemBert_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece
!pip install accelerate -U
!pip install pandas numpy matplotlib
!pip install scikit-learn seaborn
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

## Import Packages

In [2]:
from transformers import Trainer, TrainingArguments, CamembertTokenizer, CamembertForSequenceClassification, CamembertConfig
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import pipeline
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import pipeline
import pickle

## Define Functions

In [3]:
# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [4]:
 # Define pre-processing function
 class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = str(self.data.iloc[idx]['sentence'])
        label = int(self.data.iloc[idx]['encoded_labels'])

        encoding = self.tokenizer.encode_plus(
            sentence,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

## Fine-tune the CamemBert Model

In [5]:
# Load the data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/training_data.csv'
data = pd.read_csv(url)

In [6]:
# Apply LabelEncoder to labels
label_encoder = LabelEncoder()
data['encoded_labels'] = label_encoder.fit_transform(data['difficulty'])

In [7]:
# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Create empty lists for statistics
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []


# K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
    print(f"Training fold {fold+1}")
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]

    # Tokenize the datasets
    train_dataset = CustomDataset(train_data, tokenizer)
    eval_dataset = CustomDataset(val_data, tokenizer)

    # Load pre-trained model
    model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold}',
        learning_rate=0.00015,
        num_train_epochs=7,
        per_device_train_batch_size=16,
        warmup_steps=1000,
        weight_decay=0.05,
        logging_dir=f'./logs_fold_{fold}',
        logging_steps=10,
        evaluation_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=500,
        fp16=True,
    )

    # Initialize and define trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )

    # Train and evaluate the model
    trainer.train()
    eval_result = trainer.evaluate()
    accuracy_list.append(eval_result['eval_accuracy'])
    precision_list.append(eval_result['eval_precision'])
    recall_list.append(eval_result['eval_recall'])
    f1_list.append(eval_result['eval_f1'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training fold 5


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,1.759600,1.745336,0.271875,0.353052,0.271875,0.162137
200,1.369100,1.348234,0.400000,0.347845,0.400000,0.317172
300,1.047700,1.236610,0.453125,0.470326,0.453125,0.429189
400,1.072700,1.178716,0.477083,0.487083,0.477083,0.450694
500,0.816700,1.064532,0.545833,0.538302,0.545833,0.534280
600,0.879000,1.051603,0.536458,0.540424,0.536458,0.533119
700,0.879300,1.093736,0.532292,0.564423,0.532292,0.524744
800,0.747500,1.065691,0.557292,0.585308,0.557292,0.561425
900,0.790200,1.436191,0.443750,0.479044,0.443750,0.411756
1000,0.607800,1.254987,0.513542,0.568733,0.513542,0.515525


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
# Compute overall statistics of the model
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
overall_precision = sum(precision_list) / len(precision_list)
overall_recall = sum(recall_list) / len(recall_list)
overall_f1 = sum(f1_list) / len(f1_list)

print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1 Score: {overall_f1:.4f}")

Overall Accuracy: 0.5835
Overall Precision: 0.6066
Overall Recall: 0.5835
Overall F1 Score: 0.5849


## Re-train on Full Dataset

In [9]:
# Combine training and validation data
combined_data = pd.concat([train_data, val_data])

In [10]:
#Load pre-trained model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Tokenize the dataset
final_dataset = CustomDataset(combined_data, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=0.00015,
    num_train_epochs=7,
    per_device_train_batch_size=16,
    warmup_steps=1000,
    weight_decay=0.05,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="no",
    save_strategy="steps",
    save_steps=500,
    fp16=True,
    )

# Re-initialize and define the trainer
final_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset,
    compute_metrics=None
)

# Retrain the model on the whole dataset
final_trainer.train()

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Step,Training Loss
10,1.792100
20,1.790400
30,1.786300
40,1.795500
50,1.788400
60,1.782900
70,1.784600
80,1.774000
90,1.762300
100,1.756800


TrainOutput(global_step=2100, training_loss=0.7715630920728048, metrics={'train_runtime': 424.9513, 'train_samples_per_second': 79.068, 'train_steps_per_second': 4.942, 'total_flos': 2210212240588800.0, 'train_loss': 0.7715630920728048, 'epoch': 7.0})

In [11]:
# Save the final fine-tuned model and tokenizer
model.save_pretrained('./final_model')
tokenizer.save_pretrained('./final_model')

('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/sentencepiece.bpe.model',
 './final_model/added_tokens.json')

## Make Predictions

In [12]:
# Load the unlabelled data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/unlabelled_test_data.csv'
unlabelled_data = pd.read_csv(url)

In [13]:
# Load the saved fine-tuned model and tokenizer
model_path = './final_model'
model = CamembertForSequenceClassification.from_pretrained(model_path)
tokenizer = CamembertTokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
# Create a prediction pipeline
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, framework='pt', device=device)

# Predict labels for the unlabelled data
predictions = classifier(unlabelled_data['sentence'].tolist())

# Decode the numeric labels to original labels using the loaded LabelEncoder
predicted_labels = [label_encoder.inverse_transform([int(pred['label'].split('_')[-1])])[0] for pred in predictions]

# Create a new DataFrame with predictions
results_df = pd.DataFrame({
    'id': unlabelled_data['id'],
    'difficulty': predicted_labels
})

# Save the results to a new CSV file
results_df.to_csv('predicted_difficulties.csv', index=False)

print("Predictions saved to 'predicted_difficulties.csv'")


Predictions saved to 'predicted_difficulties.csv'


## Re-train on Extended Dataset

In [15]:
# Load extended data set
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/combined_random_french_sentences.csv'
full_data = pd.read_csv(url)

In [16]:
# Apply LabelEncoder to labels
label_encoder = LabelEncoder()
full_data['encoded_labels'] = label_encoder.fit_transform(full_data['difficulty'])

In [17]:
# Load pre-trained model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Tokenize dataset
full_dataset = CustomDataset(full_data, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=0.00015,
    num_train_epochs=7,
    per_device_train_batch_size=16,
    warmup_steps=1000,
    weight_decay=0.05,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='no',
    save_strategy="steps",
    save_steps=500,
    fp16=True,
    )

# Re-initialize and define the trainer
final_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_dataset,
    compute_metrics=None
)

# Retrain the model on the extended dataset
final_trainer.train()


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Step,Training Loss
10,1.790900
20,1.793400
30,1.792600
40,1.789500
50,1.780300
60,1.782200
70,1.760300
80,1.748100
90,1.728100
100,1.716300


TrainOutput(global_step=4291, training_loss=0.35101464429822066, metrics={'train_runtime': 823.9365, 'train_samples_per_second': 83.259, 'train_steps_per_second': 5.208, 'total_flos': 4512516657868800.0, 'train_loss': 0.35101464429822066, 'epoch': 7.0})

In [18]:
#save the model
model_path_full = "./fine_tuned_model_full"
model.save_pretrained(model_path_full)
tokenizer.save_pretrained(model_path_full)

('./fine_tuned_model_full/tokenizer_config.json',
 './fine_tuned_model_full/special_tokens_map.json',
 './fine_tuned_model_full/sentencepiece.bpe.model',
 './fine_tuned_model_full/added_tokens.json')

## Make Predictions

In [19]:
# Load the unlabelled data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/unlabelled_test_data.csv'
unlabelled_data = pd.read_csv(url)

In [20]:
# Load the fine-tuned model and tokenizer for predictions
model = CamembertForSequenceClassification.from_pretrained(model_path_full)
tokenizer = CamembertTokenizer.from_pretrained(model_path_full)

# Define prediction pipeline
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

# Predict the labels of the unlabelled data
predictions = classifier(unlabelled_data['sentence'].tolist())
predicted_labels = [label_encoder.inverse_transform([int(pred['label'].split('_')[-1])])[0] for pred in predictions]

# Create a new dataframe with predictions
results_df = pd.DataFrame({
    'id': unlabelled_data['id'],
    'difficulty': predicted_labels
})
results_df.to_csv('predicted_difficulties_full.csv', index=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


##Hyperparameter Optimization

In [21]:
import optuna

In [22]:
# Load the data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/training_data.csv'
data = pd.read_csv(url)

In [ ]:
# Define objective function for hyperoptimization with Optuna
def objective(trial):
    # Define hyperparameter search space
    learning_rate = trial.suggest_loguniform("learning_rate", 10e-5, 20e-5)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 7)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        logging_dir='./logs',
        logging_steps=10,
        warmup_steps=1000,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        fp16=True
    )

    # Load the tokenizer
    tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

    # Encode the labels
    label_encoder = LabelEncoder()
    data['encoded_labels'] = label_encoder.fit_transform(data['difficulty'])

    # Split the dataset into training and validation sets
    train_data, val_data = train_test_split(data, test_size=0.2, stratify=data['encoded_labels'], random_state=42)

    # Tokenize datasets
    train_dataset = CustomDataset(train_data, tokenizer)
    eval_dataset = CustomDataset(val_data, tokenizer)

    # Load the pre-trained model pre-trained
    model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

    # Initialize and define the trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )

    # Train and evaluate the model
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_accuracy"]

# Create and optimize the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# Print the best combination of parameters
print(f"Best trial accuracy: {study.best_trial.value}")
print(f"Best parameters: {study.best_trial.params}")

[I 2024-05-20 07:25:26,692] A new study created in memory with name: no-name-462f2924-9f0c-408d-801c-6c67a67daea3
<ipython-input-23-184778ba9543>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 10e-5, 20e-5)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.254700,1.145390,0.510417,0.517697,0.510417,0.503358
2,1.260000,1.192852,0.445833,0.553739,0.445833,0.406171
3,1.068300,1.204332,0.523958,0.531801,0.523958,0.506286
4,0.721100,1.182934,0.563542,0.572406,0.563542,0.558251
5,0.570700,1.455378,0.561458,0.574313,0.561458,0.562886
6,0.150600,1.941880,0.568750,0.591200,0.568750,0.572187
7,0.016300,2.151232,0.569792,0.582460,0.569792,0.572959


[I 2024-05-20 07:34:00,526] Trial 0 finished with value: 0.5697916666666667 and parameters: {'learning_rate': 0.00016168459055575829, 'per_device_train_batch_size': 8, 'num_train_epochs': 7}. Best is trial 0 with value: 0.5697916666666667.
<ipython-input-23-184778ba9543>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 10e-5, 20e-5)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to u

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.205200,1.162163,0.500000,0.486322,0.500000,0.483649
2,1.212900,1.290667,0.431250,0.508879,0.431250,0.369660
3,0.904400,1.158098,0.535417,0.543876,0.535417,0.533591
4,0.753800,1.251276,0.548958,0.559118,0.548958,0.550179
5,0.470800,1.609590,0.546875,0.564074,0.546875,0.548378
6,0.092500,1.865296,0.563542,0.588891,0.563542,0.567165


[I 2024-05-20 07:42:39,448] Trial 1 finished with value: 0.5635416666666667 and parameters: {'learning_rate': 0.00015163262020418634, 'per_device_train_batch_size': 8, 'num_train_epochs': 6}. Best is trial 0 with value: 0.5697916666666667.
<ipython-input-23-184778ba9543>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 10e-5, 20e-5)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to u

...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.364500,1.348166,0.401042,0.368140,0.401042,0.318868
2,1.110000,1.232006,0.466667,0.493078,0.466667,0.440050
3,0.866000,1.255418,0.482292,0.510091,0.482292,0.477854
4,0.812500,1.127237,0.537500,0.558742,0.537500,0.537631
5,0.559200,1.306281,0.555208,0.559220,0.555208,0.543419
6,0.362400,1.449194,0.567708,0.573144,0.567708,0.564639
7,0.126600,1.704041,0.578125,0.599406,0.578125,0.577390


[I 2024-05-20 08:26:38,667] Trial 10 finished with value: 0.578125 and parameters: {'learning_rate': 0.00010502108881020471, 'per_device_train_batch_size': 16, 'num_train_epochs': 7}. Best is trial 10 with value: 0.578125.
<ipython-input-23-184778ba9543>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 10e-5, 20e-5)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predict

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.367000,1.356315,0.420833,0.417001,0.420833,0.344600
2,1.093200,1.205986,0.465625,0.484015,0.465625,0.438395
3,0.934500,1.195978,0.509375,0.544610,0.509375,0.490381
4,0.846300,1.237566,0.543750,0.544817,0.543750,0.533999
5,0.555400,1.341948,0.561458,0.565547,0.561458,0.553779
6,0.401900,1.385425,0.576042,0.590438,0.576042,0.579963
7,0.114000,1.651396,0.569792,0.590903,0.569792,0.572529


[I 2024-05-20 08:32:35,257] Trial 11 finished with value: 0.5697916666666667 and parameters: {'learning_rate': 0.0001030286950223985, 'per_device_train_batch_size': 16, 'num_train_epochs': 7}. Best is trial 10 with value: 0.578125.
<ipython-input-23-184778ba9543>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 10e-5, 20e-5)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it fo

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.382600,1.377594,0.408333,0.381165,0.408333,0.323935
2,1.100700,1.296531,0.423958,0.456804,0.423958,0.390656
3,0.909300,1.162722,0.515625,0.523983,0.515625,0.507224
4,0.752700,1.172946,0.536458,0.535606,0.536458,0.532523
5,0.555600,1.217887,0.577083,0.578424,0.577083,0.572996
